In [2]:
!curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -o input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  3869k      0 --:--:-- --:--:-- --:--:-- 3876k


In [3]:
!ls

input.txt  test.ipynb


In [4]:
with open('input.txt', 'r') as f:
    text = f.read()
print(text[:1000])  # Print the first 1000 characters

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
print(f'The length of the text is {len(text)}')

The length of the text is 1115394


In [9]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [10]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
print(encode('hello world'))
print(decode(encode('hello world')))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [11]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [13]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [16]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [17]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f'When input is {context} the target is {target}')

When input is tensor([18]) the target is 47
When input is tensor([18, 47]) the target is 56
When input is tensor([18, 47, 56]) the target is 57
When input is tensor([18, 47, 56, 57]) the target is 58
When input is tensor([18, 47, 56, 57, 58]) the target is 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [18]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    return x, y
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-----')
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t + 1]
        target = yb[b, t]
        print(f'When input is {context.tolist()} the target is {target}')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
When input is [24] the target is 43
When input is [24, 43] the target is 58
When input is [24, 43, 58] the target is 5
When input is [24, 43, 58, 5] the target is 57
When input is [24, 43, 58, 5, 57] the target is 1
When input is [24, 43, 58, 5, 57, 1] the target is 46
When input is [24, 43, 58, 5, 57, 1, 46] the target is 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
When input is [44] the target is 53
When input is [44, 53] the target is 56
When input is [44, 53, 56] the target is 1
When input is [44, 53, 56, 1] the target is 58
When input is [44, 53, 56, 1, 58] the target 

In [30]:
# Bigram language model
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1337)
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B, T, C)
        # B, T, C = logits.shape
        # logits = logits.view(B * T, C) # (B * T, C)
        # targets = targets.view(B * T) # (B * T)
        # loss = F.cross_entropy(logits, targets)
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
            return logits, loss
        else:
            loss = None
        return logits, loss
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
        return idx
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [34]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))



xZddZAXzkIBtp3d,vAcPlgX'pM fNMLphx&flaAcL!3F.?sBiRwLTqHzot.ttRF$Fv'bL:&x&ayFVqVAHqHxv3QzteqbcUJnERZ


In [35]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [36]:
batch_size = 32
for step in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if step % 1000 == 0:
        print(f'Step {step}: loss {loss.item()}')

Step 0: loss 4.670031547546387
Step 1000: loss 3.68472957611084
Step 2000: loss 3.084817409515381
Step 3000: loss 2.6774075031280518
Step 4000: loss 2.677391290664673
Step 5000: loss 2.444664478302002
Step 6000: loss 2.6338696479797363
Step 7000: loss 2.5872673988342285
Step 8000: loss 2.434361696243286
Step 9000: loss 2.5427095890045166


In [39]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Thaf t weables mye.
Tor f beng thowithe:
Thauthin, wherure;
Fing tyoucora we d s?

Tord, g Intho heat pr f Yhzen?
Ther hurer cr il f aloulatspribr,
AG o otr thall oull
NEN theathaie

IN eruisfeveg, t wild he borong s bl?
DW:
Anto' h bes whoommeulye Werngss myou
Shar, hed wingoleshet heked
GUE t at, sours s y par 's h, de,
I hathago pe, wowhe w?

I we be lirewin ad:
Myorodeter t ceom,

I'sustier IORCENGJULI dow hof, ind hal thel becepin.
sthtour Gom:



Cannof k tigr tin mame
Wint pHandenomety rt


In [19]:
from transformers import pipeline, set_seed
import torch
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device)
generator = pipeline('text-generation', model='gpt2', device=device)
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5, truncation=True)

mps


Device set to use mps
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model, and my project will get better with time, but I think there are a lot more things that can help you"},
 {'generated_text': "Hello, I'm a language model, not a language model, so if I don't have a problem, I can fix it by creating new words"},
 {'generated_text': "Hello, I'm a language model, and I'm trying to learn some stuff. I'll try to do some basic programming and just learn better ways"},
 {'generated_text': "Hello, I'm a language model, but I don't believe in grammar. This will work for every language model. You can define it very quickly"},
 {'generated_text': 'Hello, I\'m a language model, a model of how things should be, and then we look at different things as well." I\'d like to'}]

In [17]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
print(enc.encode("Hello, I'm a language model,"))
tokens = torch.tensor(enc.encode("Hello, I'm a language model,"), dtype=torch.long)
tokens = tokens.unsqueeze(dim=0).repeat(5, 1)
print(tokens)
print(tokens.shape)
print(enc.decode(enc.encode("Hello, I'm a language model,")))


[15496, 11, 314, 1101, 257, 3303, 2746, 11]
tensor([[15496,    11,   314,  1101,   257,  3303,  2746,    11],
        [15496,    11,   314,  1101,   257,  3303,  2746,    11],
        [15496,    11,   314,  1101,   257,  3303,  2746,    11],
        [15496,    11,   314,  1101,   257,  3303,  2746,    11],
        [15496,    11,   314,  1101,   257,  3303,  2746,    11]])
torch.Size([5, 8])
Hello, I'm a language model,


In [32]:
pew = torch.arange(0, 25, dtype=torch.int32)
x = pew[:-1].view(4, -1)
y = pew[1:].view(4, -1)
print(f'x.shape: {x.shape}')
print(f'y.shape: {y.shape}')
print(f'x: {x}')
print(f'y: {y}')


x.shape: torch.Size([4, 6])
y.shape: torch.Size([4, 6])
x: tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]], dtype=torch.int32)
y: tensor([[ 1,  2,  3,  4,  5,  6],
        [ 7,  8,  9, 10, 11, 12],
        [13, 14, 15, 16, 17, 18],
        [19, 20, 21, 22, 23, 24]], dtype=torch.int32)
